# OpenAI Agent SDKとDatabricks MCPサーバーを使用したサプライチェーン・コパイロットの構築

## ソリューション概要

サプライチェーン業務において、エージェントはサービスレベルと収益に直接影響する質問を解決できます：現在の需要を満たすための在庫と生産能力はあるか？製造遅延はどこで発生し、その遅延は下流にどのように波及するか？どのワークフロー調整が混乱を最小限に抑えるか？

![Databricks MCP UI](../../images/databricks_mcp_ui.png)

このクックブックでは、OpenAI Agent SDKとDatabricks Managed MCPを使用してサプライチェーン・コパイロットを構築するプロセスを説明します。MCPにより、エージェントは在庫、売上、サプライヤーフィード、地域イベントなどの構造化および非構造化企業データをクエリして、リアルタイムの可視性、材料不足の早期検出、積極的な推奨事項を提供できます。オーケストレーション層がシステムを支え、以下を統合します：
- 構造化された在庫、需要、サプライヤーデータに対するクエリ
- 全卸売業者向けの時系列予測
- グラフベースの原材料要件と輸送最適化
- 非構造化コミュニケーション全体でのセマンティック検索を可能にするベクトルインデックス化されたメールアーカイブ
- 収益リスク計算

このガイドの最後には、分散データソースをクエリし、予測モデルを実行し、新たなボトルネックを強調し、積極的なアクションを推奨するテンプレートをデプロイします。以下のような質問に対応できます：
- L6HUK材料に依存している製品は何か？
- 予測された量のproduct autoclave_1を生産できない場合、どれだけの収益がリスクにさらされるか？
- 現在遅延している製品はどれか？
- syringe_1に遅延はあるか？
- syringe_1に必要な原材料は何か？
- 以下の原材料のいずれかに不足があるか：O4GRQ、Q5U3A、OAIFB、または58RJD？
- 卸売業者9に関連する遅延は何か？

ステークホルダーは自然言語プロンプトを送信し、即座に回答を受け取ることができます。
このガイドでは、このソリューションを独自の環境で実装するための各ステップを説明します。

## アーキテクチャ

このクックブックで紹介するアーキテクチャは、Databricksにおけるあなたの既存の分析ワークロードの上にOpenAI Agentを重ねます。Databricksコンポーネントを呼び出し可能なUnity Catalog関数として公開できます。エージェントは[OpenAI Agent SDK](https://openai.github.io/openai-agents-python/)で実装され、[Databricks Managed MCPサーバー](https://docs.databricks.com/aws/en/generative-ai/agent-framework/mcp)に接続します。

その結果、きめ細かい予測、動的な在庫推奨、サプライチェーン全体でのデータ駆動型意思決定を提供する、単一のほぼリアルタイムな対話インターフェースが実現されます。このアーキテクチャにより、既存の企業データ（構造化および非構造化）、従来のMLモデル、グラフ分析機能を活用するエージェント層が生み出されます。

![Databricks MCP Architecture](../../images/databricks_mcp_architecture.png)

## Databricks認証の設定

`~/.databrickscfg`にプロファイルを追加することで、Databricks認証を設定できます。[Databricks設定プロファイル](https://docs.databricks.com/aws/en/dev-tools/auth/config-profiles)には、Databricksが認証に必要な設定やその他の情報が含まれています。

スニペットの`WorkspaceClient(profile=...)`呼び出しがそれを取得します。これにより、保存された認証情報のうちどれを読み込むかをSDKに指示するため、コードにトークンを埋め込む必要がありません。`DATABRICKS_HOST`や`DATABRICKS_TOKEN`などの環境変数を作成する方法もありますが、`~/.databrickscfg`を使用することが推奨されます。

ワークスペースの[personal access token (PAT)](https://docs.databricks.com/aws/en/dev-tools/auth/pat#databricks-personal-access-tokens-for-workspace-users)を、Settings → Developer → Access tokens → Generate new tokenから生成し、`~/.databrickscfg`に記録してください。

このDatabricks設定プロファイルファイルを作成するには、[Databricks CLI](https://docs.databricks.com/aws/en/dev-tools/cli/)の databricks configure コマンドを実行するか、以下の手順に従ってください：
- `~/.databrickscfg`が存在しない場合は、作成してください：touch `~/.databrickscfg`
- ファイルを開いてください：`nano ~/.databrickscfg`
- ワークスペースURLとpersonal access token (PAT)を記載したプロファイルセクションを挿入してください（追加のプロファイルはいつでも追加できます）：

```bash
[DEFAULT]
host  = https://dbc-a1b2345c-d6e7.cloud.databricks.com # ここにワークスペースURLを追加
token = dapi123...    # ここにPATを追加
```

その後、Databricks CLIまたはSDKで `databricks clusters list` というサニティチェックコマンドを実行できます。認証情報の入力を求められることなくデータが返される場合、ホストが正しく、トークンが有効であることを示しています。

前提条件として、DatabricksワークスペースでServerlessコンピュートとUnity Catalogが有効になっている必要があります。

## （オプション）Databricks サプライチェーンのセットアップ

このクックブックは、独自のDatabricksサプライチェーンデータセットと分析ワークロードで使用することができます。

また、Databricksのサプライチェーン最適化ソリューションアクセラレーターのカスタマイズ版を使用することで、セットアップを加速することも可能です。これを行うには、このGitHub[リポジトリ](https://github.com/lara-openai/databricks-supply-chain)をDatabricksワークスペースにクローンし、README[ファイル](https://github.com/lara-openai/databricks-supply-chain/blob/main/README.md)の指示に従ってください。ソリューションを実行すると、エージェントが後でMCP経由でアクセスするすべてのアセット（生の企業テーブルや非構造化メールから、従来のMLモデルやグラフワークロードまで）が構築されます。

独自のデータセットとモデルを使用したい場合は、関連コンポーネントをUnity Catalog関数としてラップし、アクセラレーターで示されているようにVector Searchインデックスを定義してください。Genie Spacesを公開することも可能です。

サンプルデータは現実的な製薬ネットワークを反映しています：3つの工場が30の製品を製造し、5つの配送センターに出荷し、各配送センターは30〜60の卸売業者にサービスを提供します。リポジトリには、すべての製品-卸売業者ペアの時系列需要、配送センター-卸売業者マッピング、工場-配送センターコストマトリックス、工場の生産能力上限、および出荷遅延を示すメールアーカイブが含まれています。

![Pharma Network](../../images/pharma_network.png)

サプライチェーン運用の質問に答えるには、上流のボトルネックが生産、物流、フルフィルメントを通じてどのように連鎖するかをモデル化し、関係者がリードタイムを短縮し、過剰在庫を回避し、コストを制御できるようにする必要があります。ノートブックは、これらの生データを管理された呼び出し可能なアーティファクトに変換します：

- 需要予測と集約（[ノートブック2](https://github.com/lara-openai/databricks-supply-chain/blob/main/02_Fine_Grained_Demand_Forecasting.py)）：Holt-Winters季節モデル（または任意の好みの時系列アプローチ）を使用して、すべての卸売業者と配送センターに対して1週間先のSKU需要を生成します。Pandas UDFを使用してSparkの並列化を活用し（単一ノードのデータサイエンスコードを複数ノードに分散）、大規模な予測タスクを処理します。予測は各製品のDCレベル合計にロールアップされます。出力は配送センターレベルでの集約予測を含む`product_demand_forecasted`テーブルです。

- 原材料計画（[ノートブック3](https://github.com/lara-openai/databricks-supply-chain/blob/main/03_Derive_Raw_Material_Demand.py)）：グラフ処理を使用して製品-材料マッピングを構築し、部品表階層を通じて需要を伝播させ、大規模なコンポーネント要件を計算します。部品表をグラフに変換することで、製品予測を正確な原材料要件に変換し、`raw_material_demand`と`raw_material_supply`の2つのテーブルを生成します。

- 輸送最適化（[ノートブック4](https://github.com/lara-openai/databricks-supply-chain/blob/main/04_Optimize_Transportation.py)）：容量と需要の制約下で工場から配送センターへの輸送コストを最小化し、Pandas UDFを活用して`shipment_recommendations`に推奨事項を出力します。

- セマンティックメール検索（[ノートブック6](https://github.com/lara-openai/databricks-supply-chain/blob/main/06_Vector_Search.py)）：OpenAI埋め込みモデルを使用してサプライチェーンマネージャーのメールをベクトルインデックスに埋め込み、遅延とリスクシグナルを表面化するセマンティッククエリを可能にします。

各インサイトは[ノートブック5](https://github.com/lara-openai/databricks-supply-chain/blob/main/05_Data_Analysis_%26_Functions.py)と[ノートブック7](https://github.com/lara-openai/databricks-supply-chain/blob/main/07_More_Functions.py)でUnity Catalog（UC）関数としてラップされています（例：`product_from_raw`、`raw_from_product`、`revenue_risk`、`lookup_product_demand`、`query_unstructured_emails`）。UCはテーブル、モデル、ベクトルインデックスを同様に管理するため、エージェントは実行時に予測、BOM依存関係の追跡、収益影響の測定、履歴の取得、メール検索のいずれを行うかを決定でき、常に呼び出し元のデータアクセス権限内で動作します。

結果として、需要を予測し、原材料のギャップを特定し、物流を最適化し、隠れたリスクを表面化し、アナリストがアドホックな質問をして遅延警告を表面化できるエンドツーエンドのパイプラインが完成します。

すべてのノートブックが実行された後（ノートブック1を実行することで）、Databricks環境の準備が完了し、エージェントを構築してDatabricksに接続することができます。

## Databricks MCP サーバーに接続する

現在、[MCP 仕様](https://openai.github.io/openai-agents-python/mcp/)では、使用するトランスポートメカニズムに基づいて3種類のサーバーが定義されています：
- stdio サーバーは、アプリケーションのサブプロセスとして実行されます。これらは「ローカル」で実行されていると考えることができます。
- HTTP over SSE サーバーはリモートで実行されます。URL経由で接続します。
- Streamable HTTP サーバーは、MCP 仕様で定義されている Streamable HTTP トランスポートを使用してリモートで実行されます。

[Databricks ホスト型 MCP エンドポイント](https://docs.databricks.com/aws/en/generative-ai/agent-framework/mcp)（ベクトル検索、Unity Catalog 関数、Genie）は、標準的な HTTPS URL の背後に配置され、MCP 仕様で定義されている Streamable HTTP トランスポートを実装しています。Databricks 管理の MCP に接続できるように、ワークスペースでサーバーレスが有効になっていることを確認してください。

## Databricks MCPサーバーをOpenAI Agentに統合する

OpenAI Agentは[こちら](https://github.com/openai/openai-cookbook/blob/main/examples/mcp/building-a-supply-chain-copilot-with-agent-sdk-and-databricks-mcp/README.md)で利用できます。まず、必要な依存関係をインストールしてください：

In [ ]:
pip install -r requirements.txt

OpenAI APIに安全にアクセスするには、OpenAI APIキーが必要です。OpenAI APIを初めて使用する場合は、[アカウントにサインアップ](https://platform.openai.com/signup)してください。[こちらの手順](https://platform.openai.com/docs/libraries?project_id=proj_2NqyDkmG63zyr3TzOh64F2ac#create-and-export-an-api-key)に従って、キーを作成し、安全な場所に保存することができます。

このクックブックでは、このAgentをFastAPIで提供し、React UIを通じてチャットする方法を説明します。ただし、`main.py`は自己完結型のREPLとして設定されているため、必要な依存関係をインストールし、必要な認証情報（上記で説明したDatabricksホストとパーソナルアクセストークンを含む）を設定した後、コマンドラインから単一のコマンドでAgentを直接実行できます：

In [ ]:
python main.py

[main.py](https://github.com/openai/openai-cookbook/blob/main/examples/mcp/building-a-supply-chain-copilot-with-agent-sdk-and-databricks-mcp/main.py) ファイルは、OpenAI Agent SDKを使用してエージェントロジックを統制し、Databricks MCPベクター検索エンドポイントとUnity Catalog関数を呼び出し可能なツールとして公開します。まず、対象のカタログ、スキーマ、Unity Catalog（UC）関数パスを指す環境変数を読み込み、次に2つのツールを公開します：`vector_search`（Databricks Vector Searchインデックスをクエリする）と`uc_function`（MCPを介してUnity Catalog関数を実行する）です。両方のツールはhttpxを通じて認証されたPOSTリクエストを行い、Databricks REST APIから生のJSONを返します。両方のヘルパーは`_databricks_ctx()`ユーティリティ（DatabricksOAuthClientProviderによってサポート）を通じてワークスペースホストとPersonal Access Tokenを取得し、httpxで認証されたPOSTリクエストを発行して、生のJSON応答を返します。

`run_agent()`内では、スクリプトはサプライチェーントピックにハードスコープされた「Assistant」と呼ばれるAgentをインスタンス化します。すべての応答は登録された2つのツールのいずれかを呼び出す必要があり、ガードレールによってエージェントは物流、在庫、調達、予測以外のものを拒否するよう強制されます。各ユーザープロンプトはSDKトレースコンテキスト内で処理されます。シンプルなREPLがインタラクションを駆動します：ユーザー入力はOpenTelemetryスタイルのトレースでラップされ、Runner.runを通じてディスパッチされ、最終的な回答（またはガードレールの謝罪）が印刷されます。プログラムは`main()`内の`asyncio.run`呼び出しによって開始され、フロー全体を完全に非同期でノンブロッキングにします。

In [0]:
"""
CLI assistant that uses Databricks MCP Vector Search and UC Functions via the OpenAI Agents SDK.
"""

import asyncio
import os
import httpx
from typing import Dict, Any
from agents import Agent, Runner, function_tool, gen_trace_id, trace
from agents.exceptions import (
    InputGuardrailTripwireTriggered,
    OutputGuardrailTripwireTriggered,
)
from agents.model_settings import ModelSettings
from databricks_mcp import DatabricksOAuthClientProvider
from databricks.sdk import WorkspaceClient
from supply_chain_guardrails import supply_chain_guardrail

CATALOG = os.getenv("MCP_VECTOR_CATALOG", "main") # override catalog, schema, functions_path name if your data assets sit in a different location
SCHEMA = os.getenv("MCP_VECTOR_SCHEMA", "supply_chain_db")
FUNCTIONS_PATH = os.getenv("MCP_FUNCTIONS_PATH", "main/supply_chain_db")
DATABRICKS_PROFILE = os.getenv("DATABRICKS_PROFILE", "DEFAULT") # override if using a different profile name 
HTTP_TIMEOUT = 30.0  # seconds


async def _databricks_ctx():
    """Return (workspace, PAT token, base_url)."""
    ws = WorkspaceClient(profile=DATABRICKS_PROFILE)
    token = DatabricksOAuthClientProvider(ws).get_token()
    return ws, token, ws.config.host


@function_tool
async def vector_search(query: str) -> Dict[str, Any]:
    """Query Databricks MCP Vector Search index."""
    ws, token, base_url = await _databricks_ctx()
    url = f"{base_url}/api/2.0/mcp/vector-search/{CATALOG}/{SCHEMA}"
    headers = {"Authorization": f"Bearer {token}"}
    async with httpx.AsyncClient(timeout=HTTP_TIMEOUT) as client:
        resp = await client.post(url, json={"query": query}, headers=headers)
        resp.raise_for_status()
        return resp.json()


@function_tool
async def uc_function(function_name: str, params: Dict[str, Any]) -> Dict[str, Any]:
    """Invoke a Databricks Unity Catalog function with parameters."""
    ws, token, base_url = await _databricks_ctx()
    url = f"{base_url}/api/2.0/mcp/functions/{FUNCTIONS_PATH}"
    headers = {"Authorization": f"Bearer {token}"}
    payload = {"function": function_name, "params": params}
    async with httpx.AsyncClient(timeout=HTTP_TIMEOUT) as client:
        resp = await client.post(url, json=payload, headers=headers)
        resp.raise_for_status()
        return resp.json()


async def run_agent():
    agent = Agent(
        name="Assistant",
        instructions="You are a supply-chain assistant for Databricks MCP; you must answer **only** questions that are **strictly** about supply-chain data, logistics, inventory, procurement, demand forecasting, etc; for every answer you must call one of the registered tools; if the user asks anything not related to supply chain, reply **exactly** with 'Sorry, I can only help with supply-chain questions'.",
        tools=[vector_search, uc_function],
        model_settings=ModelSettings(model="gpt-4o", tool_choice="required"),
        output_guardrails=[supply_chain_guardrail],
    )

    print("Databricks MCP assistant ready. Type a question or 'exit' to quit.")

    while True:
        user_input = input("You: ").strip()
        if user_input.lower() in {"exit", "quit"}:
            break

        trace_id = gen_trace_id()
        with trace(workflow_name="Databricks MCP Agent", trace_id=trace_id):
            try:
                result = await Runner.run(starting_agent=agent, input=user_input)
                print("Assistant:", result.final_output)
            except InputGuardrailTripwireTriggered:
                print("Assistant: Sorry, I can only help with supply-chain questions.")
            except OutputGuardrailTripwireTriggered:
                print("Assistant: Sorry, I can only help with supply-chain questions.")


def main():
    asyncio.run(run_agent())


if __name__ == "__main__":
    main()

[databricks_mcp.py](https://github.com/openai/openai-cookbook/blob/main/examples/mcp/building-a-supply-chain-copilot-with-agent-sdk-and-databricks-mcp/databricks_mcp.py) は、集約された認証抽象化として機能します。このファイルは、先ほど作成したPersonal Access Tokenを指定されたWorkspaceClient（ws.config.token）から取得し、アプリケーションの残りの部分をDatabricks固有のOAuthロジックから保護します。すべてのトークン処理の詳細をこの単一のモジュールに限定することで、Databricksの認証スキームに将来的な変更があった場合でも、このファイルを更新するだけで対応できます。

In [ ]:
"""
Databricks OAuth client provider for MCP servers.
"""

class DatabricksOAuthClientProvider:
    def __init__(self, ws):
        self.ws = ws

    def get_token(self):
        # For Databricks SDK >=0.57.0, token is available as ws.config.token
        return self.ws.config.token

[supply_chain_guardrails.py](https://github.com/openai/openai-cookbook/blob/main/examples/mcp/building-a-supply-chain-copilot-with-agent-sdk-and-databricks-mcp/supply_chain_guardrails.py) は、候補となる回答を分類する第二のエージェント（「サプライチェーンチェック」）を起動することで、軽量な出力ガードレールを実装しています。メインエージェントは下書きの返答をこのチェッカーに渡し、チェッカーはブール値の`is_supply_chain`を含むPydanticオブジェクトを返します。このフラグが`false`の場合、ガードレールがトリップワイヤーを発動し、呼び出し元は拒否応答に置き換えます。

In [ ]:
"""
Output guardrail that blocks answers not related to supply-chain topics.
"""
from __future__ import annotations

from pydantic import BaseModel
from agents import Agent, Runner, GuardrailFunctionOutput
from agents import output_guardrail
from agents.run_context import RunContextWrapper

class SupplyChainCheckOutput(BaseModel):
    reasoning: str
    is_supply_chain: bool


guardrail_agent = Agent(
    name="Supply-chain check",
    instructions=(
        "Check if the text is within the domain of supply-chain analytics and operations "
        "Return JSON strictly matching the SupplyChainCheckOutput schema"
    ),
    output_type=SupplyChainCheckOutput,
)


@output_guardrail
async def supply_chain_guardrail(
    ctx: RunContextWrapper, agent: Agent, output
) -> GuardrailFunctionOutput:
    """Output guardrail that blocks non-supply-chain answers"""
    text = output if isinstance(output, str) else getattr(output, "response", str(output))
    result = await Runner.run(guardrail_agent, text, context=ctx.context)
    return GuardrailFunctionOutput(
        output_info=result.final_output,
        tripwire_triggered=not result.final_output.is_supply_chain,
    )

## FastAPIでエージェントを提供する

バックエンド（Fast API）を起動するには、以下のコマンドを実行してください：

In [ ]:
python -m uvicorn api_server:app --reload --port 8000

APIは http://localhost:8000 で利用可能になります（FastAPIのドキュメントは http://localhost:8000/docs で確認できます）。

[api_server.py](https://github.com/openai/openai-cookbook/blob/main/examples/mcp/building-a-supply-chain-copilot-with-agent-sdk-and-databricks-mcp/api_server.py) は、エージェントをストリーミング /chat APIエンドポイントとして公開するFastAPIバックエンドです。起動時にCORSを設定してローカルフロントエンドが通信できるようにし、その後 `build_mcp_servers()` を定義します。この関数は呼び出し元のDatabricksワークスペースに認証し、2つのHTTP「サーバーツール」（1つはベクトル検索用、もう1つはUnity-Catalog関数用）を構築し、低レイテンシ使用のために事前接続します。/chat への各POSTリクエストには単一のユーザーメッセージが含まれます。ハンドラーは、mcp_serversリストにこれらのストリーミングツールが設定され、モデルが毎ターンツールを呼び出すよう強制された新しいAgentを起動します。

In [0]:
"""
FastAPI wrapper that exposes the agent as a streaming `/chat` endpoint.
"""

import os
import asyncio
import logging
from fastapi import FastAPI
from fastapi.responses import StreamingResponse
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
from agents.exceptions import (
    InputGuardrailTripwireTriggered,
    OutputGuardrailTripwireTriggered,
)
from agents import Agent, Runner, gen_trace_id, trace
from agents.mcp import MCPServerStreamableHttp, MCPServerStreamableHttpParams
from agents.model_settings import ModelSettings
from databricks_mcp import DatabricksOAuthClientProvider
from databricks.sdk import WorkspaceClient

from supply_chain_guardrails import supply_chain_guardrail

CATALOG = os.getenv("MCP_VECTOR_CATALOG", "main")
SCHEMA = os.getenv("MCP_VECTOR_SCHEMA", "supply_chain_db")
FUNCTIONS_PATH = os.getenv("MCP_FUNCTIONS_PATH", "main/supply_chain_db")
DATABRICKS_PROFILE = os.getenv("DATABRICKS_PROFILE", "DEFAULT")
HTTP_TIMEOUT = 30.0  # seconds

app = FastAPI()

# Allow local dev front‑end
app.add_middleware(
    CORSMiddleware,
    allow_origins=["http://localhost:5173"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

class ChatRequest(BaseModel):
    message: str


async def build_mcp_servers():
    """Initialise Databricks MCP vector & UC‑function servers."""
    ws = WorkspaceClient(profile=DATABRICKS_PROFILE)
    token = DatabricksOAuthClientProvider(ws).get_token()

    base = ws.config.host
    vector_url = f"{base}/api/2.0/mcp/vector-search/{CATALOG}/{SCHEMA}"
    fn_url = f"{base}/api/2.0/mcp/functions/{FUNCTIONS_PATH}"

    async def _proxy_tool(request_json: dict, url: str):
        import httpx

        headers = {"Authorization": f"Bearer {token}"}
        async with httpx.AsyncClient(timeout=HTTP_TIMEOUT) as client:
            resp = await client.post(url, json=request_json, headers=headers)
            resp.raise_for_status()
            return resp.json()

    headers = {"Authorization": f"Bearer {token}"}

    servers = [
        MCPServerStreamableHttp(
            MCPServerStreamableHttpParams(
                url=vector_url,
                headers=headers,
                timeout=HTTP_TIMEOUT,
            ),
            name="vector_search",
            client_session_timeout_seconds=60,
        ),
        MCPServerStreamableHttp(
            MCPServerStreamableHttpParams(
                url=fn_url,
                headers=headers,
                timeout=HTTP_TIMEOUT,
            ),
            name="uc_functions",
            client_session_timeout_seconds=60,
        ),
    ]

    # Ensure servers are initialized before use
    await asyncio.gather(*(s.connect() for s in servers))
    return servers


@app.post("/chat")
async def chat_endpoint(req: ChatRequest):
    try:
        servers = await build_mcp_servers()

        agent = Agent(
            name="Assistant",
            instructions="Use the tools to answer the questions.",
            mcp_servers=servers,
            model_settings=ModelSettings(tool_choice="required"),
            output_guardrails=[supply_chain_guardrail],
        )

        trace_id = gen_trace_id()

        async def agent_stream():
            logging.info(f"[AGENT_STREAM] Input message: {req.message}")
            try:
                with trace(workflow_name="Databricks MCP Example", trace_id=trace_id):
                    result = await Runner.run(starting_agent=agent, input=req.message)
                    logging.info(f"[AGENT_STREAM] Raw agent result: {result}")
                    try:
                        logging.info(
                            f"[AGENT_STREAM] RunResult __dict__: {getattr(result, '__dict__', str(result))}"
                        )
                        raw_responses = getattr(result, "raw_responses", None)
                        logging.info(f"[AGENT_STREAM] RunResult raw_responses: {raw_responses}")
                    except Exception as log_exc:
                        logging.warning(f"[AGENT_STREAM] Could not log RunResult details: {log_exc}")
                    yield result.final_output
            except InputGuardrailTripwireTriggered:
                # Off-topic question denied by guardrail
                yield "Sorry, I can only help with supply-chain questions."
            except OutputGuardrailTripwireTriggered:
                # Out-of-scope answer blocked by guardrail
                yield "Sorry, I can only help with supply-chain questions."
            except Exception:
                logging.exception("[AGENT_STREAM] Exception during agent run")
                yield "[ERROR] Exception during agent run. Check backend logs for details."

        return StreamingResponse(agent_stream(), media_type="text/plain")

    except Exception:
        logging.exception("chat_endpoint failed")
        return StreamingResponse(
            (line.encode() for line in ["Internal server error 🙈"]),
            media_type="text/plain",
            status_code=500,
        )

エンドポイントは、エージェントが推論してMCPツールを呼び出している間、ブラウザにトークンをストリーミングで返します。

## React チャット UI を通じてユーザーとエンゲージする

別のターミナルで、以下を実行してフロントエンド（React UI）を起動してください：

In [ ]:
cd ui
npm install
npm run dev

アプリは http://localhost:5173 で利用可能になります

[/ui フォルダ](https://github.com/openai/openai-cookbook/blob/main/examples/mcp/building-a-supply-chain-copilot-with-agent-sdk-and-databricks-mcp/ui)のReactチャットUIは、バックエンドエージェントとやり取りするためのユーザーフレンドリーなWebインターフェースを提供します。会話履歴を表示するコンポーネントと、メッセージを送信するためのテキスト入力機能を備えています。

ユーザーがメッセージを送信すると、UIはそれをバックエンドの /chat エンドポイントに送信し、エージェントの応答をリアルタイムでストリーミングし、新しいコンテンツが到着するたびにチャットウィンドウを更新します。この設計は会話体験を重視しており、ユーザーが質問をして、レスポンシブでインタラクティブなWebアプリケーション内で、Databricksを活用したエージェントから回答を受け取ることを簡単にしています。

特に、[ChatUI.jsx](https://github.com/openai/openai-cookbook/blob/main/examples/mcp/building-a-supply-chain-copilot-with-agent-sdk-and-databricks-mcp/ui/src/components/ChatUI.jsx)ファイルには、ユーザーメッセージがバックエンドに送信される方法や、エージェントからのストリーミング応答がリアルタイムで処理・表示される方法など、チャットインターフェースの中核となるロジックが含まれています。

In [ ]:
"""
Code snippet handling the token stream coming from the FastAPI /chat endpoint.
"""
const reader = response.body.getReader();
while (true) {
  const { done, value } = await reader.read();
  if (done) break;
  assistantMsg.text += new TextDecoder().decode(value);
  setMessages(m => {
    const copy = [...m];
    copy[copy.length - 1] = { ...assistantMsg };
    return copy;
  });
}

UIはエージェントの応答が到着すると同時にストリーミングして表示し、スムーズでリアルタイムなチャット体験を作り出します。これを強調することで、UIがバックエンドエージェントからインタラクティブで会話的なフィードバックをどのように実現しているかを読者に明確に示すことができます。

![Databricks MCP UI](../../images/databricks_mcp_ui.png)

## アプリにプロンプトを入力する

http://localhost:5173 にアクセスして、以下のプロンプトを試してみてください：
- L6HUK材料に依存している製品は何ですか？
- autoclave_1の予測生産量を生産できない場合、どの程度の収益がリスクにさらされますか？
- 現在遅延している製品はありますか？
- syringe_1に遅延はありますか？
- syringe_1に必要な原材料は何ですか？
- 次の原材料のいずれかに不足はありますか：O4GRQ、Q5U3A、OAIFB、または58RJD？
- 卸売業者9に関連する遅延は何ですか？

エージェントは関連するツールを呼び出し、ユーザーに対して根拠のある回答をフォーマットします。

## OpenAI API ダッシュボードでエージェントの呼び出しをトレースする

OpenAI API [ダッシュボード](https://platform.openai.com/prompts)では、Tracesビューを開いてエージェントが呼び出したすべての関数を確認できます。以下の例では、エージェントは最初に`raw_from_product`を呼び出して特定の製品にリンクされた材料を取得し、次に`revenue_risk`を呼び出して不足による収益への影響を推定しています。

![Tracing Dashboard](../../images/tracing_dashboard_databricks_mcp.png)

## 次のステップ

* マルチターン機能の追加を検討できます
* 独自のワークスペースにこのセットアップを適応させたい場合は、Genie Space MCPサーバーを追加することもできます

## 参考資料
- Databricks Managed MCP [ドキュメント](https://docs.databricks.com/aws/en/generative-ai/agent-framework/mcp)
- OpenAI Agent SDK [ドキュメント](https://openai.github.io/openai-agents-python/)
- OpenAI Agent Guardrails [ドキュメント](https://openai.github.io/openai-agents-python/guardrails/)
- Openai-agents-python サンプル [コードスニペット](https://github.com/openai/openai-agents-python/tree/main/examples/mcp/streamablehttp_example)